In [294]:
# from google.colab import drive
# drive.mount('/content/drive')

In [295]:
# !nvidia-smi

In [296]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline

In [297]:
tf.random.set_seed(1)

In [298]:
california = fetch_california_housing()
feature_names = california.feature_names
data = california.data
df = pd.DataFrame(data, columns=feature_names)
df['target'] = california.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [299]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
 8   target      20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [300]:
df.shape

(20640, 9)

In [301]:
y = df['target'].copy()
X = df.drop('target', axis=1).copy()

In [302]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [303]:
tf_X_train, tf_X_test, tf_y_train, tf_y_test = train_test_split(X, y, test_size = 0.25, random_state=13)
print(tf_X_train.shape, type(tf_X_train))
print(tf_X_test.shape, type(tf_X_test))
print(tf_y_train.shape, type(tf_y_train))
print(tf_y_test.shape, type(tf_y_test))

(15480, 8) <class 'numpy.ndarray'>
(5160, 8) <class 'numpy.ndarray'>
(15480,) <class 'pandas.core.series.Series'>
(5160,) <class 'pandas.core.series.Series'>


In [304]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(8, 64)
        self.layer2 = nn.Linear(64, 64)
        self.bn = nn.BatchNorm1d(64)
        self.dp = nn.Dropout(0.25)
        self.out = nn.Linear(64, 1)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.dp(x)
        x = self.bn(x)
        x = self.out(x)
        return x

net = Net()

In [305]:
for i in range(len(list(net.parameters()))):
  print(list(net.parameters())[i].shape)

torch.Size([64, 8])
torch.Size([64])
torch.Size([64, 64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([1, 64])
torch.Size([1])


In [306]:
class MyCalifornia(torch.utils.data.Dataset):
    
    def __init__(self,df_train, y_train):
        
        self.x=torch.tensor(df_train).type(torch.float32)
        self.y = torch.tensor(np.array(y_train)).type(torch.float32)
        self.n_samples= df_train.shape[0]
        
    
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples

In [307]:
train_dataset =  MyCalifornia(tf_X_train, tf_y_train)

In [308]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True, num_workers=2)

In [309]:
test_dataset = MyCalifornia(tf_X_test, tf_y_test)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
                                          shuffle=False)

In [310]:
optimizer_1 = [torch.optim.Adam(net.parameters(), lr=0.01),"Adam"]
optimizer_2 = [torch.optim.RMSprop(net.parameters(), lr=0.01, alpha=0.99),"RMSprop"]
optimizer_4 = [torch.optim.Adagrad(net.parameters(), lr=0.01), "Adagrad"]
optimizer_3 = [torch.optim.SGD(net.parameters(), lr=0.01),"SGD"]


# optimizer =  torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [311]:
num_epochs = 10
valtable = []
for optimizer in [optimizer_1, optimizer_2,optimizer_4,optimizer_3]:
    print("\n\t",optimizer[1])
    optimizer=optimizer[0]
    for epoch in range(num_epochs):
      running_loss, running_items = 0.0, 0.0
      for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)

        if i % 500 == 0:
          net.eval()
          print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}', end='. ')
          running_loss, running_items = 0.0, 0.0
          
          test_running_right, test_running_total = 0.0, 0.0
          for i, data in enumerate(test_loader):
            test_outputs = net(data[0])
            def MSELoss(yhat,y):
                return torch.mean((yhat-y)**2)

            criter = MSELoss
            test_running_total += len(data[1])
            mse = criter(test_outputs,data[1])
            test_running_right += mse

          print(f"test MSE {test_running_right / test_running_total:.3f}")





	 Adam
Epoch [1/10]. Step [1/242]. Loss: 0.093. 

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test MSE 0.181


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([56])) that is different to the input size (torch.Size([56, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/10]. Step [1/242]. Loss: 0.020. test MSE 0.043
Epoch [3/10]. Step [1/242]. Loss: 0.029. test MSE 0.043
Epoch [4/10]. Step [1/242]. Loss: 0.019. test MSE 0.042
Epoch [5/10]. Step [1/242]. Loss: 0.021. test MSE 0.043
Epoch [6/10]. Step [1/242]. Loss: 0.027. test MSE 0.043
Epoch [7/10]. Step [1/242]. Loss: 0.020. test MSE 0.042
Epoch [8/10]. Step [1/242]. Loss: 0.016. test MSE 0.042
Epoch [9/10]. Step [1/242]. Loss: 0.019. test MSE 0.043
Epoch [10/10]. Step [1/242]. Loss: 0.020. test MSE 0.042

	 RMSprop
Epoch [1/10]. Step [1/242]. Loss: 0.019. test MSE 0.064
Epoch [2/10]. Step [1/242]. Loss: 0.018. test MSE 0.043
Epoch [3/10]. Step [1/242]. Loss: 0.024. test MSE 0.042
Epoch [4/10]. Step [1/242]. Loss: 0.015. test MSE 0.043
Epoch [5/10]. Step [1/242]. Loss: 0.028. test MSE 0.043
Epoch [6/10]. Step [1/242]. Loss: 0.023. test MSE 0.042
Epoch [7/10]. Step [1/242]. Loss: 0.017. test MSE 0.044
Epoch [8/10]. Step [1/242]. Loss: 0.024. test MSE 0.043
Epoch [9/10]. Step [1/242]. Loss: 0.